# Assignment 5 - Data Crawling from Website

이번 과제에서 학생들은 웹사이트에서 데이터를 수집하고 원하는 형태로 정리하는 연습을 하게될 것이다.

* 제출 방법: GitHub (11/25 자정까지)
* 파일 이름: A5-학번-이름(영어로).zip 형식으로 (예: A5-13403-999-jiyoon.zip)

## 1. 미디어다음에서 제공하는 영어사전 웹사이트에서 랜돔으로 100개의 단어를 수집하고, 단어와 단어의 의미를 보기 좋게 출력하시오.

예: http://dic.daum.net/word/view.do?wordid=ekw000078303&q=help


* 저장 형식은 자유. 단, 가능한 한 보기 좋게 내용을 편집하여 저장하자.

  예)
  
    단어 1 
      1. 뜻풀이  
      2. 뜻풀이 
    ===  
    단어 2 
      1. 뜻풀이

In [ ]:
#랜덤 url list 100개 만들기
from random import random, choice

url_list = []
for i in range(100):
    a = str(round(random(), 5))
    b = choice([0,1])
    url_list.append("http://dic.daum.net/word/view.do?wordid=ekw{}{}{}".format("000", b, a[2:]))

In [ ]:
#단어와 뜻을 찾아서 저장
from bs4 import BeautifulSoup
import urllib.request
import json
from collections import OrderedDict

word_list = []
for base_url in url_list:
    
    wordss = OrderedDict()
    with urllib.request.urlopen(base_url) as url:
        doc = url.read()
        soup = BeautifulSoup(doc, "html.parser")
        words = soup.find_all("h3", class_="tit_cleanword")
        means = soup.find_all("span", class_="txt_mean")   
        for word in words:
            
            wordss["단어:"] = word.span.text.strip()
        for mean in means:
            wordss["뜻:"] = mean.text
        
    word_list.append(wordss)
    
with open('A5_words.json', 'w', encoding="utf-8") as make_file:
    json.dump(word_list, make_file, ensure_ascii=False, indent="\t")

## 2. 트위터에서 키워드를 중심으로 데이터를 수집하고 텍스트를 분석하자.

* 텍스트 분석은 frequency, co-occurrence 등 지난 시간에 다루었던 것들을 이용해 보자.
* 이번 과제는 
    * 트위터 수집 코드
    * 수집된 데이터
    * 수집된 데이터를 분석하는 코드
    
    등으로 이루어질 것이다. 모든 결과물을 zip으로 압축해서 제출한다.
* 이번 과제는 굳이 jupyter notebook의 형태로 제출하지 않아도 된다. 아마 jupyter notebook을 사용한다면 <수집된 데이터를 분석하는 코드> 정도가 있을 것이다.

In [ ]:
#수집된 데이터를 읽어오기

import json

json_data = []
with open("tweet_stream.json", encoding = "UTF-8") as file:
    data = file.readlines()
    for d in data:
        json_data.append(json.loads(d))

tweet_text = []
i = 0
for tweet in json_data:
    if ("text" in tweet) and ("user" in tweet):
        tweet_text.append(str(i) + " - " + tweet["user"]["name"] + " :: " + tweet["text"])
        i = i + 1

In [ ]:
#명사 추출

from konlpy.tag import Komoran
tagger = Komoran()
tagged_tweet = [tagger.pos(tweet) for tweet in tweet_text]

noun_list = []
for sent in tagged_tweet:    
    for word, tag in sent:
        if tag in ['NNP', 'NNG']:
            noun_list.append(word)
            
from collections import Counter
tweet_noun_counts = Counter(noun_list)
tweet_noun_counts

In [ ]:
#wordcloud 그리기

%matplotlib inline
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import numpy as np
from PIL import Image

twit_mask = np.array(Image.open("twitter_mask.png"))
twit_mask

cloud = WordCloud(width=900, height=600, colormap="Accent", mask=twit_mask,
                  font_path='C:\Windows\Fonts\HoonSaemaulundongR.ttf',
                  background_color='white')

cloud = cloud.fit_words(tweet_noun_counts)

plt.figure(figsize=(15, 20))
plt.axis('off')
plt.imshow(cloud)
plt.show()